# Training and Validation

In [6]:
import pandas as pd

df = pd.read_csv("../data/concat.csv")
df

,edit_history_tweet_ids,id,text,label,clean_text
0,['1576050330114363392'],1576050330114363392,@joicehasselmann Bolsonaro NÃO quer aproveitad...,-1,Bolsonaro NÃO quer aproveitadores como você.
1,['1576050330110160901'],1576050330110160901,o bolsonaro é um anjo enviado de deus para nos...,1,o bolsonaro é um anjo enviado de deus para nos...
2,['1576050318852640769'],1576050318852640769,@AndradeRNegro2 @Loucosdovalle Bolsonaro é con...,1,Bolsonaro é contra o aborto e defende a vida...
3,['1576050318596399104'],1576050318596399104,@Victorrmsk É a milésima vez que ele fala ao v...,0,É a milésima vez que ele fala ao vivo que não...
4,['1576050309822287873'],1576050309822287873,"@OGloboPolitica Bolsonaro é tão fake, q até o ...",-1,"Bolsonaro é tão fake, q até o laranja q ele l..."
...,...,...,...,...,...
234,['1576039456552853504'],1576039456552853504,a pessoa não tem nem idade pra votar e ainda q...,-1,a pessoa não tem nem idade pra votar e ainda q...
235,['1576039448760156161'],1576039448760156161,A corrupção de Lula não causou estrago só no B...,-1,A corrupção de Lula não causou estrago só no B...
236,['1576039448093265920'],1576039448093265920,eu n aguento mais essa gnt falando de política...,-1,eu n aguento mais essa gnt falando de política...
237,['1576039446331281409'],1576039446331281409,@Emanuel_Jrz @jairbolsonaro @MariaFdaCabal O B...,1,O Brasil não quer mais essa gangue na polít...


In [7]:
import numpy as np

inputs = df['text'].to_numpy()
target = df['label'].to_numpy()

In [9]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('../distiluse-base-multilingual-cased-v1')
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
  (2): Dense({'in_features': 768, 'out_features': 512, 'bias': True, 'activation_function': 'torch.nn.modules.activation.Tanh'})
)

In [10]:
embeddings = model.encode(inputs)

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(embeddings, target, test_size=0.2, stratify = target, random_state=42)

In [12]:
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import classification_report

eclf1 = VotingClassifier(estimators=[('ABC', AdaBoostClassifier(n_estimators=500)), ('GBC', GradientBoostingClassifier(n_estimators=500))], voting='soft')
eclf1 = eclf1.fit(x_train, y_train)

pred = eclf1.predict(x_val)

print(classification_report(y_val, pred))

              precision    recall  f1-score   support

          -1       0.58      0.75      0.65        20
           0       0.50      0.38      0.43        16
           1       0.30      0.25      0.27        12

    accuracy                           0.50        48
   macro avg       0.46      0.46      0.45        48
weighted avg       0.48      0.50      0.48        48



# Test data

In [13]:
x = ['Bolsonaro é o melhor presidente possível', 'Lula sempre deu o seu melhor pelo país', 'Ciro coitado, não serve pra nada', 'Todos os candidatos são ruins', 'Gostei muito dessa proposta', 'Bolsonaro foi ao congresso', 'Essa eleição é uma piada mesmo', 'Simplesmente horrível política', 'Não consigo entender como tem gente que não gosta de política no Brasil', 'Estou muito contente com o meu canditato!', 'Não acho é nada sobre isso', 'Sério que você acha o lula bom? kkkkkkkkkkkkk', "Ele é um ladrão velho", "Eleições são uma perda de tempo mesmo", 'Qualquer candidato eleito vai continuar a mesma coisa', '5 dias pro Lula ganhar no primeiro turno em galera', 'meu pai parou de acreditar em tudo que vê na internet quando eu fiz uma montagem com uma foto dele dizendo que ele era foragido da polícia, pra provar que qualquer um pode inventar algo', 'insuportável essa época de política', 'Vai deixar de gostar do Neymar por conta de posição política? Eu nunca gostei do Neymar.']

In [14]:
embeddings_test = model.encode(x)

In [15]:
y_pred = eclf1.predict(embeddings_test)
y_pred

array([ 1,  1, -1, -1,  1,  0, -1, -1, -1,  1,  0,  0, -1, -1,  0,  1,  0,
       -1, -1], dtype=int64)

In [17]:
df_results = pd.DataFrame({'Text' : x, 'label' : y_pred})
df_results

,Text,label
0,Bolsonaro é o melhor presidente possível,1
1,Lula sempre deu o seu melhor pelo país,1
2,"Ciro coitado, não serve pra nada",-1
3,Todos os candidatos são ruins,-1
4,Gostei muito dessa proposta,1
5,Bolsonaro foi ao congresso,0
6,Essa eleição é uma piada mesmo,-1
7,Simplesmente horrível política,-1
8,Não consigo entender como tem gente que não go...,-1
9,Estou muito contente com o meu canditato!,1
